In [1]:
from math import radians, cos, sin, asin, sqrt, ceil, floor
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import smopy
from tqdm.notebook import tqdm

In [2]:
data_path = r'U:\My Documents\Projekte\AI4Mobile\Data\Berlin'

df = pd.read_parquet(data_path+r'\cellular_dataframe.parquet')
df = df.dropna(subset=['Latitude', 'Longitude', 'Pos in Ref Round'])
df['datarate'] = df['datarate']/1e6 # transform datarate in from bit/s to Mbit/s

# add PCell_SNR_max col
def max_combine(row, key):
    return max(row[key+'_1'], row[key+'_2'])

df['PCell_SNR_max'] = df.apply(max_combine, axis=1, args=('PCell_SNR',))

In [3]:
# configuration of plots

box_size = 50 # in meters

plt.rcParams['figure.figsize'] = (8, 6)

dpi = 500
font_size = 16
s = 8 # size of squares in plot

In [4]:
for (direction, target_datarate), group_df in df.groupby(['direction', 'target_datarate']):
    print('%s %d kbit/s : %d values' % (direction, target_datarate/1e3, len(group_df)))

downlink 400 kbit/s : 31345 values
downlink 350000 kbit/s : 57905 values
uplink 400 kbit/s : 41868 values
uplink 75000 kbit/s : 59665 values


In [5]:
# generate list of columns to plot

phy = ['RSRP', 'RSRQ', 'RSSI', 'SNR']

col_list = [f'PCell_{p}_1' for p in phy] + [f'PCell_{p}_2' for p in phy] + [f'PCell_{p}_max' for p in phy]
col_list = col_list + ['datarate', 'ping_ms', 'num_values']

col_labels = {'PCell_RSRP_1': 'RSRP_1 [dBm]',
              'PCell_RSRQ_1': 'RSRQ_1 [dB]',
              'PCell_RSSI_1': 'RSSI_1 [dBm]',
              'PCell_SNR_1': 'SNR [dB]',
              'PCell_RSRP_2': 'RSRP_2 [dBm]',
              'PCell_RSRQ_2': 'RSRQ_2 [dB]',
              'PCell_RSSI_2': 'RSSI_2 [dBm]',
              'PCell_SNR_2': 'SNR [dB]',
              'PCell_RSRP_max': 'RSRP_max [dBm]',
              'PCell_RSRQ_max': 'RSRQ_max [dB]',
              'PCell_RSSI_max': 'RSSI_max [dBm]',
              'PCell_SNR_max': 'SNR [dB]',
              'datarate': 'Datarate [Mbit/s]',
              'ping_ms': 'Ping [ms]',
              'num_values': 'Number of values per tile'}

col_list

['PCell_RSRP_1',
 'PCell_RSRQ_1',
 'PCell_RSSI_1',
 'PCell_SNR_1',
 'PCell_RSRP_2',
 'PCell_RSRQ_2',
 'PCell_RSSI_2',
 'PCell_SNR_2',
 'PCell_RSRP_max',
 'PCell_RSRQ_max',
 'PCell_RSSI_max',
 'PCell_SNR_max',
 'datarate',
 'ping_ms',
 'num_values']

In [6]:
def haversine(lon1, lat1, lon2, lat2):
    """
    Calculate the great circle distance in kilometers between two points 
    on the earth (specified in decimal degrees)
    """
    # convert decimal degrees to radians 
    lon1, lat1, lon2, lat2 = map(radians, [lon1, lat1, lon2, lat2])

    # haversine formula 
    dlon = lon2 - lon1 
    dlat = lat2 - lat1 
    a = sin(dlat/2)**2 + cos(lat1) * cos(lat2) * sin(dlon/2)**2
    c = 2 * asin(sqrt(a)) 
    r = 6371 # Radius of earth in kilometers. Use 3956 for miles. Determines return value units.
    return c * r

In [7]:
lon_min = df['Longitude'].min() # x
lon_max = df['Longitude'].max()
lat_min = df['Latitude'].min() # y
lat_max = df['Latitude'].max()

lon_extent = lon_max - lon_min
lat_extent = lat_max - lat_min

# number of boxes in each direction
lon_num = ceil(haversine(lon_min, (lat_min+lat_max)/2, lon_max, (lat_min+lat_max)/2) * 1000 / box_size)
lat_num = ceil(haversine((lon_min+lon_max)/2, lat_min, (lon_min+lon_max)/2, lat_max) * 1000 / box_size)

# assign each sample 2D indexes for the box it lies
df['lon_idx'] = np.floor((df['Longitude'] - lon_min) / lon_extent * lon_num).astype(int)
df['lat_idx'] = np.floor((df['Latitude'] - lat_min) / lat_extent * lat_num).astype(int)

df['box_lon'] = ((df['lon_idx']+0.5) / lon_num) * lon_extent + lon_min
df['box_lat'] = ((df['lat_idx']+0.5) / lat_num) * lat_extent + lat_min

In [8]:
def create_heatmap(col, direction, op, fmt, show_plots, min_values_per_box):
    
    direction_df = df.loc[df['direction'] == direction]
    direction_df = direction_df.loc[direction_df['operator'] == op]
    if col == 'datarate':
        direction_df = direction_df.loc[direction_df['target_datarate'] != 0.4]
    elif col == 'ping_ms':
        direction_df = direction_df.loc[direction_df['target_datarate'] == 0.4]
    
    # collect all values
    box_longitudes = []
    box_latitudes  = []
    box_values = []
    for (lon_idx, lat_idx), group_df in direction_df.groupby(['lon_idx', 'lat_idx']):
        
        # make sure that there are sufficient samples in each box
        if len(group_df) < min_values_per_box:
            continue
        
        box_longitudes.append(lon_min + (lon_idx+0.5) * (lon_extent/lon_num))
        box_latitudes.append(lat_min + (lat_idx+0.5) * (lat_extent/lat_num))
        if col == 'num_values':
            box_values.append(len(group_df))
        else:
            box_values.append(group_df[col].mean())

    # generate background
    mp = smopy.Map((lat_min, lon_min, lat_max, lon_max), z=14, margin=0.0)
    ax = mp.show_mpl(dpi=dpi)

    # draw on the map
    x, y = mp.to_pixels(np.array(box_latitudes), np.array(box_longitudes))
    if direction == 'downlink' and col == 'datarate':
        vmax = 100
        print('vmax=%d' % vmax)
    elif col == 'ping_ms':
        vmax = 100
        print('vmax=%d ms' % vmax)
    else:
        vmax = None
    
    im = ax.scatter(x, y, marker='s', c=box_values, vmax=vmax, s=s, alpha=0.9, cmap='plasma') # aggregated REM
    cb = plt.colorbar(im, pad=0.02, orientation='horizontal')
    cb.set_label(col_labels[col], rotation=0, labelpad=17, fontsize=font_size)
    cb.ax.tick_params(labelsize=font_size)

    plt.tight_layout()
    plt.savefig('figures\heatmap_operator%d_%s_%s.%s' % (op, col, direction, fmt))
    if not show_plots:
        plt.close()

In [9]:
fmt = 'pdf' # format for saving
show_plots = False
min_values_per_box = 10

for c in tqdm(col_list):
    for operater in [1, 2]:
        create_heatmap(c, 'downlink', operater, fmt, show_plots, min_values_per_box)
        create_heatmap(c, 'uplink', operater, fmt, show_plots, min_values_per_box)

  0%|          | 0/15 [00:00<?, ?it/s]

PCell_SNR_max
